List Minio bucket and create a file with the elements present in the bucket

In [ ]:
from minio import Minio
from minio.error import S3Error
import math

def get_token(text):
    browser=['token":"','","file_stage_in']
    pos=[]
    for n in browser:
       k =text.find(n)
       if k!=-1:
          pos.append(k)
       else:
          break
    if k==-1:
       print('Error in conection')
       return None
    else:
       return text[pos[0]+8:pos[1]]

def get_cpuService(text):
   browser=['cpu":"','","total_memory']
   pos=[]
   for n in browser:
       k=text.find(n)
       if k!=-1:
          pos.append(k)
       else:
          break
   if k==-1:
       print('Error in conection')
       return None
   else:
       return 1000*float(text[pos[0]+6:pos[1]])
       


from minio import Minio
from minio.error import S3Error

# MinIO client configuration
client = Minio(
    "",  # MinIO server
    access_key="",
    secret_key="",
    secure=True
)

# Bucket name and folder prefix
bucket_name = ""
folder_prefix = "input/"

# Name of the file where the object names will be saved
output_file = "index.txt"
output_path = "input/index.txt"

try:
    # List the objects in the bucket that end with '.jpg'
    objects = client.list_objects(bucket_name, prefix=folder_prefix)
    
    # Filter and write the names of objects that end in '.jpg'
    object_list = []
    for obj in objects:
        if obj.object_name.endswith('.jpg'):
            object_list.append(obj.object_name)

    # Show the amount of images
    num_imag = len(object_list)
    print(f"{num_imag} images found.")

    # Open the file in write mode
    print(f"Names of the objects saved in {output_file}")
    with open(output_file, 'w') as file:
        for obj_name in object_list:
            print(f"{obj_name}\n")
            file.write(f"{obj_name}\n")
    
    # Upload the text file to the bucket
    client.fput_object(
        bucket_name,
        output_path,
        output_file,
        content_type="text/plain"
    )
    print(f"Archivo {output_file} subido a {bucket_name}")

except S3Error as exc:
    print("Error occurred: ", exc)


Node states of a cluster in terms of CPU

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# URL to which you want to make the request
url = ""

# Authentication credentials
username = ""
password = ""

cpu_Alloc=0

# Make the GET request with basic authentication
response = requests.get(url, auth=HTTPBasicAuth(username, password))

# Check the status of the response
if response.status_code == 200:
      
    # Convert the response to JSON
    try:
        data = response.json()
        
        # Verify that the response is an array of objects
        if isinstance(data, list):
            nodos=len(data)
            # Iterate over each object in the array
            if nodos > 1:
                # Iterate over each object in the array, except the first one
                for obj in data[1:]:
                    print(f"cpuAlloc: {int(obj['cpuCapacity']) - int(obj['cpuUsage'])}")
                    cpu_Alloc+=int(obj['cpuCapacity']) - int(obj['cpuUsage'])
        else:
            print("The response is not a JSON array of objects.")
    
    except ValueError as e:
        print("Error converting the response to JSON:", e)
else:
    print(f"Request error: {response.status_code}")
    print("Error message:")
    print(response.text)
print(cpu_Alloc)


Calculate the range of images to send in each service invocation.

In [ ]:
# search for the CPU needed to run the service defined in its creation (fdl)

url1 = ""
response = requests.get(url1, auth=HTTPBasicAuth(username, password))

if response.status_code == 200:
   resp=response.text
   print(resp)
   cpu_service = get_cpuService(resp)
   print(cpu_service)
   token=get_token(resp)
   print(token)
else:
    print(f"Request error: {response.status_code}")
    print("Error message:")
    print(response.text)

# Calculate the number of service invocations according to the CPU available in the cluster and the service's CPU
cant_invoke=math.floor(cpu_Alloc/cpu_service)
print(cant_invoke)

Service invocation with (curl u oscar-py) with image range as input parameters to the service.

In [ ]:
import requests

resto = (num_imag - 1) % cant_invoke
img_invoke = int(num_imag / cant_invoke)
print(f"Images per invocation: {img_invoke}")
#print(resto)

headers = {    
    'Authorization': "Bearer " + token,
    'Content-Type': 'application/json',
}

for i in range(cant_invoke):
    inicio = i * img_invoke + 1
    fin = (i + 1) * img_invoke
    if i == cant_invoke - 1:
        fin = fin + resto
        
    data = {
        "start": inicio,
        "end": fin
    }
    try:
        response = requests.post('', headers=headers, json=data) # for async use /job/
        if response.status_code == 200:
            print("Services OK")
        else:
            print(response.status_code)
            
    except requests.exceptions.RequestException as ex:
        print("Error running service: ", ex)
    finally:
        if response is not None:
            print(response.text)


In [ ]:
headers = {    
    'Authorization': "Bearer " + token,
    'Content-Type': 'application/json',
}
print(headers)

# POST request
data = '{"message": "Hi there"}'

try:
    response = requests.post('', headers=headers, data=data)
    if response.status_code == 200:
       print("Services OK")
       #print(response.text)
except Exception as ex:
    print("Error running service: ", ex)
